In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('merged_data_fillna_timeid.csv')
df

,Date,Time,Id,ResidualNoWinsorCumReturn,RawNoWinsorCumReturn,estVol,SharesOutstanding,MDV_63,CleanMid,CumVolume,IsOpen,ResidualNoWinsorCumReturn winsorized,RawNoWinsorCumReturn winsorized,estVol winsorized,SharesOutstanding winsorized,MDV_63 winsorized,CleanMid winsorized,CumVolume winsorized
0,20140102,10:00:00.000,IDZQ0003GF7,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0,1.0,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0
1,20140102,10:00:00.000,IDZQ00047M8,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0,1.0,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0
2,20140102,10:00:00.000,IDZQ00048R1,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0,1.0,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0
3,20140102,10:00:00.000,IDZQ0005R34,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0,1.0,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0
4,20140102,10:00:00.000,IDZQ0005SC2,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0,1.0,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446039,20171229,17:30:00.000,ID00GVXK059,-0.002841,-0.002123,0.249899,727.820,6115657.0,6.719321,501410.0,1.0,-0.002841,-0.002123,0.249899,727.820,6115657.0,6.719321,501410.0
2446040,20171229,17:30:00.000,ID00HFWT7M4,0.000843,-0.005238,0.104843,170.875,55438068.0,234.749070,18108.0,1.0,0.000843,-0.005238,0.104843,170.875,55438068.0,216.695569,18108.0
2446041,20171229,17:30:00.000,ID00HRW4917,0.033315,0.031948,0.192046,27.126,2810998.5,58.018875,13716.0,1.0,0.024701,0.031163,0.192046,27.126,2810998.5,58.018875,13716.0
2446042,20171229,17:30:00.000,ID00HTQM455,0.011946,0.009285,0.130896,100.000,3763925.0,53.096570,3374.0,1.0,0.011946,0.009285,0.130896,100.000,3763925.0,53.096570,3374.0


In [37]:
def compute_features(df):
    # intraday return features
    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17res'
    features = temp.reset_index()

    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id') - df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # 17:30 ma raw return features
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = '17ma1raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = '17ma3raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = '17ma5raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='RawNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = '17ma20raw'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    features['17ma1-3raw'] = features['17ma1raw'] - features['17ma3raw']
    features['17ma1-5raw'] = features['17ma1raw'] - features['17ma5raw']
    features['17ma1-20raw'] = features['17ma1raw'] - features['17ma20raw']

    # 17:30 ma residual return features
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = '17ma1res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = '17ma3res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = '17ma5res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='ResidualNoWinsorCumReturn winsorized',index='Date',columns='Id').rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = '17ma20res'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    features['17ma1-3res'] = features['17ma1res'] - features['17ma3res']
    features['17ma1-5res'] = features['17ma1res'] - features['17ma5res']
    features['17ma1-20res'] = features['17ma1res'] - features['17ma20res']

    # market value
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id') * df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'market_value'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # intraday volume features
    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '10/17vol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='16:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') - df[df['Time']=='10:00:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')) / df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = '16/17vol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # turnover ma
    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=1).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma1'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=3).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma3'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=5).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma5'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = (df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id') / df[df['Time']=='17:30:00.000'].pivot_table(values='SharesOutstanding winsorized',index='Date',columns='Id')).rolling(window=20).mean()
    temp = temp.unstack()
    temp.name = 'turnover_ma20'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # original features 
    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='estVol winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'estVol'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid17'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='16:00:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid16'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='10:00:00.000'].pivot_table(values='CleanMid winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'cleanMid10'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    temp = df[df['Time']=='17:30:00.000'].pivot_table(values='CumVolume winsorized',index='Date',columns='Id')
    temp = temp.unstack()
    temp.name = 'vol17'
    temp = temp.reset_index()
    features = features.merge(temp,on=['Id','Date'])

    # Normalization
    features.iloc[:,2:] = (features.iloc[:,2:] - features.iloc[:,2:].mean())/features.iloc[:,2:].std()

    # Merging with target
    target = df[df["Time"] == "17:30:00.000"][['Date','Id','ResidualNoWinsorCumReturn winsorized']].copy()
    merged = features.merge(target ,on =['Date','Id']).drop_duplicates()
    merged = merged.sort_values(['Id','Date'],ascending = [True, True])
    merged["y"] = merged.groupby(['Id'])["ResidualNoWinsorCumReturn winsorized"].shift(-1)
    merged = merged.dropna(how = 'any')
    merged = merged.drop(["Id", "Date","ResidualNoWinsorCumReturn winsorized"], axis = 1)
    return merged

In [38]:
m = compute_features(df)
m

,10/17res,10/17raw,16/17res,16/17raw,17ma1raw,17ma3raw,17ma5raw,17ma20raw,17ma1-3raw,17ma1-5raw,...,turnover_ma1,turnover_ma3,turnover_ma5,turnover_ma20,estVol,cleanMid17,cleanMid16,cleanMid10,vol17,y
19,-1.577538,0.345493,0.861773,0.266602,-0.170505,-0.473889,1.645554,0.363198,0.129827,-1.008020,...,0.515209,1.155765,2.703089,2.681067,1.350592,-0.112409,-0.111202,-0.108973,-0.346780,-0.017619
20,1.551047,0.263254,0.251149,0.087380,-0.256243,-0.444490,0.297564,0.301799,0.004034,-0.432008,...,0.148677,0.370587,2.227437,2.711021,1.273906,-0.122133,-0.121744,-0.119533,-0.419927,0.015128
21,-0.710095,-0.356051,1.235010,-0.128711,0.219830,-0.118281,-0.385348,0.357955,0.353221,0.436331,...,-0.179171,0.177581,0.713336,2.696084,1.362486,-0.121670,-0.122247,-0.125022,-0.485355,0.008872
22,-1.263018,-0.819749,-1.617720,-0.216271,0.527603,0.280336,-0.012979,0.802465,0.444993,0.592479,...,-0.389804,-0.158106,0.096656,2.637186,1.394780,-0.112914,-0.113893,-0.120780,-0.527390,0.024245
23,-1.306220,-1.182957,-0.881210,-0.442774,0.660045,0.803534,0.437925,1.003195,0.233521,0.514952,...,-0.072985,-0.240325,0.000471,2.467033,1.404955,-0.105130,-0.107152,-0.116596,-0.464164,0.002732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814812,-0.701946,-0.671947,0.903348,-0.149156,0.117419,0.470120,-0.212623,0.343894,-0.191883,0.236682,...,-0.865900,-0.929720,-0.939415,-0.987459,-0.208830,1.270922,1.269062,1.253346,-0.633637,0.000437
814813,-0.050997,0.322561,0.030365,0.176715,-0.078323,0.467259,-0.074792,-0.001497,-0.429131,-0.049167,...,-0.865900,-0.962101,-0.970507,-1.000944,-0.079971,1.270922,1.270876,1.270834,-0.633637,-0.003103
814814,-0.270579,-0.199407,0.616036,0.676473,-0.121161,-0.046991,0.278939,0.083054,-0.114507,-0.272828,...,-0.795177,-0.939515,-0.971134,-1.011400,-0.183450,1.264199,1.272565,1.259164,-0.630692,-0.009896
814815,1.073508,1.032164,-0.674091,-1.088851,-0.871502,-0.611661,-0.076872,-0.093289,-0.628800,-0.928356,...,-0.735147,-0.891016,-0.961042,-1.013960,-0.133837,1.236449,1.223192,1.263942,-0.628192,0.001759


In [35]:
m[m['Id']=='ID000BC0991'][['Date','y']]

,Date,y
19,20140129,-0.017619
20,20140130,0.015128
21,20140131,0.008872
22,20140203,0.024245
23,20140204,0.002732
...,...,...
629,20160805,0.005452
630,20160808,-0.014010
631,20160809,-0.001101
632,20160810,-0.001561


In [36]:
df[(df['Id']=='ID000BC0991')&(df['Time']== "17:30:00.000")][['Date','ResidualNoWinsorCumReturn winsorized']].tail(10)

,Date,ResidualNoWinsorCumReturn winsorized
1576784,20160801,-0.004413
1579172,20160802,-0.001458
1581568,20160803,0.006847
1583968,20160804,0.003267
1586364,20160805,-0.003280
1588754,20160808,0.005452
1591142,20160809,-0.014010
1593532,20160810,-0.001101
1595923,20160811,-0.001561
1598310,20160812,0.000719


In [10]:
f = f.iloc[:,2:].dropna()
(f - f.mean())/f.std()

,17ma1raw,17ma3raw,17ma5raw,17ma20raw,17ma1-3raw,17ma1-5raw,17ma1-20raw,17ma1res,17ma3res,17ma5res,...,16/17vol,turnover_ma1,turnover_ma3,turnover_ma5,turnover_ma20,estVol,cleanMid17,cleanMid16,cleanMid10,vol17
19,-0.171481,-0.476375,1.649008,0.363200,0.130078,-1.010617,-0.255091,0.591758,-0.151479,3.197606,...,0.139022,0.503273,1.143141,2.689620,2.681026,1.408061,-0.116495,-0.115271,-0.113034,-0.360798
20,-0.257379,-0.446902,0.297826,0.301802,0.004086,-0.433706,-0.329846,-1.505300,-1.389295,0.849444,...,-0.283873,0.136487,0.358254,2.214357,2.710980,1.329729,-0.126260,-0.125857,-0.123639,-0.433131
21,0.219581,-0.119875,-0.386703,0.357957,0.353826,0.435988,0.147294,1.292326,0.221538,-0.214993,...,0.717195,-0.191589,0.165321,0.701494,2.696042,1.420210,-0.125796,-0.126364,-0.129152,-0.497829
22,0.527928,0.279742,-0.013453,0.802462,0.445743,0.592380,0.366803,0.757864,0.318718,-0.147891,...,-0.256603,-0.402369,-0.170242,0.085319,2.637145,1.453198,-0.117002,-0.117974,-0.124891,-0.539396
23,0.660617,0.804252,0.438520,1.003190,0.233936,0.514732,0.459205,2.071273,2.411204,1.459784,...,-0.521418,-0.085329,-0.252430,-0.010788,2.466993,1.463591,-0.109185,-0.111204,-0.120690,-0.476874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281333,-0.079128,0.467133,-0.075412,-0.001491,-0.429765,-0.050268,-0.080863,0.037266,0.352724,0.207035,...,-3.688246,-0.878796,-0.973938,-0.980972,-1.000953,-0.053210,1.272742,1.272724,1.272692,-0.644458
1281334,-0.122046,-0.048406,0.279157,0.083059,-0.114643,-0.274278,-0.143322,-0.265134,0.083486,0.252039,...,-0.783369,-0.808023,-0.951361,-0.981598,-1.011409,-0.158910,1.265991,1.274420,1.260972,-0.641546
1281335,-0.873784,-0.614492,-0.077496,-0.093282,-0.629750,-0.930829,-0.876203,-0.845464,-0.628027,-0.231294,...,-0.762829,-0.747952,-0.902880,-0.971515,-1.013968,-0.108232,1.238122,1.224836,1.265771,-0.639075
1281336,-0.134138,-0.645923,-0.485310,-0.114974,0.296602,0.092626,-0.112576,0.150215,-0.561946,-0.251705,...,-0.202751,-0.821568,-0.881676,-0.960526,-1.015045,-0.225237,1.240169,1.208215,1.241949,-0.642104
